In [1]:
import os
import pytesseract

In [2]:
import json
import cv2
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from pytesseract import Output

In [4]:
%matplotlib inline

In [5]:
json_path = './6_1_4_predict.json'
img_path = './data_450/6_1_4.jpg'

In [6]:
with open(json_path) as json_file:
    data = json.load(json_file)

In [7]:
for text_line in data['text_lines'][:5]:
    print(text_line)

{'x': 1041, 'y': 2306, 'w': 83, 'h': 28}
{'x': 415, 'y': 2306, 'w': 81, 'h': 26}
{'x': 101, 'y': 2306, 'w': 275, 'h': 28}
{'x': 1490, 'y': 2305, 'w': 154, 'h': 28}
{'x': 726, 'y': 2304, 'w': 265, 'h': 32}


In [8]:
class JsonReader(object):
    
    def __init__(self, json_path, img_path):
        self.text_image = cv2.imread(img_path)
        self.image_dir = img_path
        self.json_path = json_path
        self.list_coordinate = []
        self.json_data = {}
        self.readJsonFile()
        self.initJsonData()
        
    def initJsonData(self):
        self.json_data['model_name'] = self.json_data['model_name']
        self.json_data['image_dir'] = self.image_dir
        self.json_data['text_lines'] = []
        
    def readJsonFile(self):
        with open(self.json_path) as json_file:
            self.json_data = json.load(json_file)
            
    def createListCoordinate(self):
        for text_line in data['text_lines']:
            self.list_coordinate.append(text_line)
        
    def crop_text_line(self, coordinate, pixel_padding = 0):
        x = int(coordinate['x']) - pixel_padding
        y = int(coordinate['y']) - pixel_padding
        h = int(coordinate['h']) + pixel_padding
        w = int(coordinate['w']) + pixel_padding
        
        text_line = self.text_image[y:y+h, x:x+w]
        
        # call Tesseract
        text = pytesseract.image_to_string(text_line, lang='vie', config='--psm 7 --oem 1 tsv')
        text_line_coor = {}
        text_line_coor['coordinate'] = coordinate
        text_line_coor['text_content'] = text
        data = pytesseract.image_to_data(text_line, lang='vie', config='--psm 7 --oem 1 tsv', output_type=Output.DATAFRAME)
        
        list_words = [] 
        for index, row in data[data["conf"] > 0].iterrows():
            coordinate_word = {}
            word_data = {}
            coordinate_word['x'] = row['left']
            coordinate_word['y'] = row['top']
            coordinate_word['w'] = row['width']
            coordinate_word['h'] = row['height']
            word_data['word'] = row['text']
            word_data['w_coor'] = coordinate_word
            list_words.append(word_data)
            
        text_line_coor['list_words'] = list_words
        
        self.json_data['text_lines'].append(text_line_coor)
    
    def processOCR(self, padding=0, json_save_name='text.json'):
        self.createListCoordinate()
        for coordinate in self.list_coordinate:
            self.crop_text_line(coordinate, padding)
        
        with open(json_save_name, 'w', encoding='utf-8') as write_file:
            json.dump(self.json_data, write_file, indent=4, ensure_ascii=False)

In [9]:
JsonData = JsonReader(json_path, img_path)

In [10]:
JsonData.processOCR(2, 'text.json')